# CLG Step 3: Network Construction & Analysis

This notebook processes the calibrated single-neuron traces to construct functional connectivity networks and analyze their topology.

**Workflow:**
1.  **Load Data**: Import the calibrated traces from Step 2.
2.  **Preprocessing**:
    *   **Denoising (PCA)**: Use Principal Component Analysis to reconstruct traces and reduce noise.
    *   **dF/F Calculation**: Compute relative fluorescence changes.
3.  **Network Construction**:
    *   Compute Pairwise Pearson Correlation Matrix.
    *   Define Edges based on a high correlation threshold (e.g., > 0.95).
4.  **Network Analysis**:
    *   Compute topological metrics: **Degree**, **Eigenvector Centrality**, **Clustering Coefficient**.
    *   Compare distributions.
5.  **Visualization**: Plot metric distributions.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, squareform
import networkx as nx
from joblib import Parallel, delayed

# Optional: Import AllenSDK for dF/F if available
try:
    import allensdk.brain_observatory.dff as dff
    HAS_ALLEN = True
except ImportError:
    print("AllenSDK not found. A simple dF/F fallback will be used.")
    HAS_ALLEN = False

# --- CONFIGURATION ---
INPUT_FILE = r'Z:\GDZ\zebrafish_ptz\20220317fish4\spon\extraction_results\calibrated_traces.npz'
PCA_VARIANCE_THRESHOLD = 0.95
CORRELATION_THRESHOLD = 0.95
SAMPLING_RATE = 1.0 # Hz (e.g. 1Hz for zebrafish whole brain)

print(f"Loading data from: {INPUT_FILE}")

## 1. Load Calibrated Data

In [ ]:
data = np.load(INPUT_FILE)
traces = data['traces']
positions = data['positions']
ids = data['ids']

print(f"Loaded {traces.shape[0]} neurons with {traces.shape[1]} timepoints.")

# Filter out rows that are all zeros (dead cells or extraction errors)
valid_mask = np.any(traces != 0, axis=1)
traces = traces[valid_mask]
positions = positions[valid_mask]
ids = ids[valid_mask]

print(f"After cleaning zeros: {traces.shape[0]} neurons remain.")

## 2. Preprocessing: dF/F & PCA Denoising

In [ ]:
def compute_dff_simple(traces, window=600):
    """Fallback sliding window median dF/F"""
    # Simplified for demo; ideally use AllenSDK
    # This is a placeholder. Real implementation should handle baselining properly.
    return (traces - np.mean(traces, axis=1, keepdims=True)) / (np.mean(traces, axis=1, keepdims=True) + 1e-6)

def compute_dff_allen_wrapper(traces, fs):
    if not HAS_ALLEN:
        return compute_dff_simple(traces)
    
    # AllenSDK dF/F parameters
    # long window ~ 600s, short window ~ 3s
    median_kernel_long = int(600 * fs // 2 * 2 + 1)
    median_kernel_short = int((10/3) * fs // 2 * 2 + 1)
    
    # Processing in chunks to save memory if needed, here we do it directly for simplicity
    # or wrap in Parallel loop as in your original script
    dff_traces = dff.compute_dff_windowed_median(
        traces, 
        median_kernel_long=median_kernel_long,
        median_kernel_short=median_kernel_short
    )
    return dff_traces

print("Computing dF/F...")
# Using joblib for parallel processing if list of traces is large
# For simplicity, we assume single batch here or use the parallel wrapper defined below
traces_dff = compute_dff_allen_wrapper(traces, SAMPLING_RATE)
print("dF/F calculation complete.")

# --- PCA Denoising ---
def pca_reconstruction(data, variance_threshold=0.95):
    print("Running PCA Denoising...")
    # Standardize
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data.T).T # Sklearn PCA expects (n_samples, n_features). 
    # Note: Neural data usually (Neurons x Time). 
    # If we treat Time as samples -> fit(traces.T)
    # If we treat Neurons as samples (to find population modes) -> fit(traces)
    # Your original code used fit(data_processed), check orientation.
    # Assuming we want to denoise temporal dynamics of neurons using population modes:
    
    # Common approach: PCA on (Time x Neurons) to find temporal components
    pca = PCA(n_components=variance_threshold, svd_solver='full')
    
    # Fit on Time x Neurons
    data_t = data.T
    data_transformed = pca.fit_transform(data_t)
    
    print(f"Selected {pca.n_components_} components explaining {np.sum(pca.explained_variance_ratio_)*100:.2f}% variance.")
    
    # Inverse transform to reconstruct denoised data
    data_reconstructed_t = pca.inverse_transform(data_transformed)
    return data_reconstructed_t.T # Return to (Neurons x Time)

traces_denoised = pca_reconstruction(traces_dff, PCA_VARIANCE_THRESHOLD)

## 3. Network Construction
Compute the correlation matrix and define the graph.

In [ ]:
print("Computing Correlation Matrix...")
# pdist computes pairwise distances between observations (Neurons). 
# Metric='correlation' computes 1 - correlation.
# We want the Correlation itself.
dist_matrix = pdist(traces_denoised, metric='correlation')
corr_matrix = 1 - squareform(dist_matrix)

# Remove diagonal (self-correlation)
np.fill_diagonal(corr_matrix, 0)

# Thresholding
print(f"Building Graph with Threshold > {CORRELATION_THRESHOLD}...")
adj_matrix = np.abs(corr_matrix) > CORRELATION_THRESHOLD

# Build NetworkX Graph
G = nx.from_numpy_array(adj_matrix)
print(f"Graph constructed: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.")

# Extract Largest Connected Component (Optional but recommended)
largest_cc = max(nx.connected_components(G), key=len)
G_sub = G.subgraph(largest_cc).copy()
print(f"Largest Connected Component: {G_sub.number_of_nodes()} nodes.")

## 4. Network Metrics & Analysis
Calculate Degree, Eigenvector Centrality, etc.

In [ ]:
print("Calculating Network Metrics...")

# 1. Degree
degrees = dict(G_sub.degree())
degree_values = list(degrees.values())

# 2. Eigenvector Centrality (can be slow for large graphs)
try:
    eigen_centrality = nx.eigenvector_centrality(G_sub, max_iter=1000)
    eigen_values = list(eigen_centrality.values())
except Exception as e:
    print(f"Eigenvector centrality failed to converge: {e}")
    eigen_values = []

# 3. Clustering Coefficient (Optional, computationally intensive)
# clustering = nx.clustering(G_sub)
# clust_values = list(clustering.values())

print("Metrics calculation complete.")

## 5. Visualization
Plot the Degree Distribution.

In [ ]:
plt.figure(figsize=(10, 6))

# Log-Log Degree Distribution
plt.hist(degree_values, bins=50, density=True, alpha=0.7, color='skyblue', edgecolor='black')
plt.yscale('log')
plt.xscale('log')
plt.title("Degree Distribution (Log-Log)")
plt.xlabel("Degree")
plt.ylabel("Probability")
plt.grid(True, which="both", ls="--", alpha=0.5)
plt.show()

# Plot Eigenvector Centrality Distribution
if eigen_values:
    plt.figure(figsize=(10, 6))
    plt.hist(eigen_values, bins=50, density=True, alpha=0.7, color='orange', edgecolor='black')
    plt.yscale('log')
    plt.title("Eigenvector Centrality Distribution")
    plt.xlabel("Centrality")
    plt.ylabel("Frequency")
    plt.show()